<a href="https://colab.research.google.com/github/kapibara3chi/NLPTesForGC/blob/main/NLPTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pip

In [2]:
!pip  install -U ginza ja_ginza_electra

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 5.2 MB/s 
     |████████████████████████████████| 2.2 MB 37.8 MB/s 
     |████████████████████████████████| 283 kB 57.9 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 4.7 MB 12.3 MB/s 
     |████████████████████████████████| 1.1 MB 24.6 MB/s 
     |████████████████████████████████| 7.6 MB 41.7 MB/s 
     |████████████████████████████████| 182 kB 43.7 MB/s 
     |████████████████████████████████| 6.6 MB 26.3 MB/s 
  Created wheel for SudachiDict-core: filename=SudachiDict_core-20221021-py3-none-any.whl size=71574782 sha256=87c3a9eded6b4bd5602510ebd50defce25d9e74f9771ebfbb7d9b0458408c9c8
  Stored in directory: /root/.cache/pip/wheels/ca/e3/ed/e78fecf6fd34349114d292242a16fc08d513fb32c2d9c5d786
  Created wheel for sudachitra: filename=SudachiTra-0.1.7-py3-none-any.whl size=266095 sha256=93c5d

# main

---

## algol
- 名詞の親をたどっていく
- 過程の動詞を抽出
- 名詞にたどり着くまで探す

In [3]:
import ginza
from spacy import displacy
import spacy
class DependencyAnalysis:
    def __init__(self):
        self.nlp = spacy.load('ja_ginza_electra')
        self.indices = []

    def get_analysis(self, text):
        """係り受け解析"""
        doc = self.nlp(text)

        token_head_list = []

        for sent in doc.sents:
            for token in sent:
                token_head_list.append(
                    {"i": token.i, "orth": token.orth_, "base": token.lemma_,
                     "head": token.head.i, "dep": token.dep_,"pos":token.pos_,
                     "tag":token.tag_})

        displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

        return token_head_list
    # def get(self, text, target):
    #   # 係り受け解析
    #   self.depend_indices = self.get_analysis(text)
    #   print(self.depend_indices)

    def get_words(self,  m, head):
        """対象の領域の抽出"""
        
        if m["head"] == head :
            for n in self.depend_indices:
                self.get_words(m=n, head=m["i"])
                            
            self.indices.append(m["i"])
        
        return
    def get(self, text, target):
        
        # 係り受け解析
        self.depend_indices = self.get_analysis(text)
        print(self.depend_indices)
                
        # 係り元のインデックスを取得
        head = self.depend_indices[target]["head"]
        # print(head)

        # 係り元のかかり先を探索
        self.indices.append(head)
        print(self.indices)#list of head?
        for m in self.depend_indices:
            if m["head"] == head and m["dep"] != "ROOT":
                self.get_words(m, head=head)

        clause = ""
        for clause_num in sorted(self.indices):
            clause = clause + str(self.depend_indices[clause_num]["orth"])
            
        return clause

if __name__ == '__main__':
  dependency = DependencyAnalysis()
  txt="下流あごを2mmだすと分離は改善するが、コンタミが悪化する。xx75では良化する。"
  # ret=dependency.get_analysis("下流あごをだすと分離は改善するが、コンタミが悪化する")
  # for i in ret:
    # print(i)
  ret=dependency.get(txt,0)
  print(ret)


error megagonlabs/transformers-ud-japanese-electra-base-ginza-510 does not appear to have a file named config.json.
error We couldn't connect to 'https://huggingface.co' to load this model, couldn't find it in the cached files and it looks like /tmp/tmpek81emga/config.json is not the path to a directory containing a config.json file.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.
trying to download model from huggingface hub: megagonlabs/transformers-ud-japanese-electra-base-ginza-510 ...


succeded
/usr/local/lib/python3.8/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[{'i': 0, 'orth': '下流', 'base': '下流', 'head': 1, 'dep': 'compound', 'pos': 'NOUN', 'tag': '名詞-普通名詞-一般'}, {'i': 1, 'orth': 'あご', 'base': 'あご', 'head': 5, 'dep': 'obj', 'pos': 'NOUN', 'tag': '名詞-普通名詞-一般'}, {'i': 2, 'orth': 'を', 'base': 'を', 'head': 1, 'dep': 'case', 'pos': 'ADP', 'tag': '助詞-格助詞'}, {'i': 3, 'orth': '2', 'base': '2', 'head': 4, 'dep': 'nummod', 'pos': 'NUM', 'tag': '名詞-数詞'}, {'i': 4, 'orth': 'mm', 'base': 'mm', 'head': 5, 'dep': 'obl', 'pos': 'NOUN', 'tag': '名詞-普通名詞-助数詞可能'}, {'i': 5, 'orth': 'だす', 'base': 'だす', 'head': 9, 'dep': 'advcl', 'pos': 'VERB', 'tag': '動詞-非自立可能'}, {'i': 6, 'orth': 'と', 'base': 'と', 'head': 5, 'dep': 'mark', 'pos': 'SCONJ', 'tag': '助詞-接続助詞'}, {'i': 7, 'orth': '分離', 'base': '分離', 'head': 9, 'dep': 'nsubj', 'pos': 'NOUN', 'tag': '名詞-普通名詞-サ変可能'}, {'i': 8, 'orth': 'は', 'base': 'は', 'head': 7, 'dep': 'case', 'pos': 'ADP', 'tag': '助詞-係助詞'}, {'i': 9, 'orth': '改善', 'base': '改善', 'head': 15, 'dep': 'advcl', 'pos': 'VERB', 'tag': '名詞-普通名詞-サ変可能'}, {'i': 10, 'o

## Universal POS Tags version2
[URL](https://www.jstage.jst.go.jp/article/jnlp/26/1/26_3/_pdf/-char/ja)

|Tag  | 品詞| 
| -- | --|
ADJ|形容詞
ADV|副詞
VERB|動詞
NOUN|名詞
PROPN|固有名詞
NUM|数詞


|Label| 依存関係| 
| -- | --|
obj
advcl
nsubj|名詞主語
nummod
compound
case